In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# Set the path to the data
data_dir = '/content/drive/My Drive/SRI'
csv_file = '/content/drive/My Drive/SRI/english.csv'

In [ ]:
# Load the CSV file
data = pd.read_csv(csv_file)

In [ ]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(data['image'], data['label'], test_size=0.2, random_state=42)


In [ ]:
# Preprocess the data
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_dataframe(
    dataframe=data.loc[X_train.index],
    directory=data_dir,
    x_col="image",
    y_col="label",
    target_size=(400, 300),
    batch_size=64,
    class_mode='categorical'
)

Found 2728 validated image filenames belonging to 62 classes.


In [ ]:
val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_dataframe(
    dataframe=data.loc[X_val.index],
    directory=data_dir,
    x_col="image",
    y_col="label",
    target_size=(400, 300),
    batch_size=64,
    class_mode='categorical'
)

Found 682 validated image filenames belonging to 62 classes.


In [ ]:
# Load the pre-trained DenseNet121 model
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(400, 300, 3))


29084464/29084464 [==============================] - 0s 0us/step


In [ ]:
# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.2)(x)
output = Dense(62, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=output)

In [ ]:
# Compile the model
model.compile(optimizer=Adam(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the model with early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
model.fit(train_generator, epochs=100, validation_data=val_generator, callbacks=[early_stopping])


Epoch 1/100
43/43 [==============================] - 1241s 28s/step - loss: 3.7763 - accuracy: 0.1136 - val_loss: 3.1487 - val_accuracy: 0.3343
Epoch 2/100
43/43 [==============================] - 61s 1s/step - loss: 2.7601 - accuracy: 0.3849 - val_loss: 2.4094 - val_accuracy: 0.5806
Epoch 3/100
43/43 [==============================] - 61s 1s/step - loss: 2.1663 - accuracy: 0.5652 - val_loss: 1.9722 - val_accuracy: 0.6481
Epoch 4/100
43/43 [==============================] - 62s 1s/step - loss: 1.7783 - accuracy: 0.6598 - val_loss: 1.6534 - val_accuracy: 0.6994
Epoch 5/100
43/43 [==============================] - 62s 1s/step - loss: 1.5150 - accuracy: 0.7064 - val_loss: 1.4916 - val_accuracy: 0.7185
Epoch 6/100
43/43 [==============================] - 61s 1s/step - loss: 1.3248 - accuracy: 0.7357 - val_loss: 1.3324 - val_accuracy: 0.7126
Epoch 7/100
43/43 [==============================] - 62s 1s/step - loss: 1.1835 - accuracy: 0.7636 - val_loss: 1.2080 - val_accuracy: 0.7610
Epoch 8/10

In [ ]:
# Evaluate the model on the test set
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(
    dataframe=data.loc[X_val.index],
    directory=data_dir,
    x_col="image",
    y_col="label",
    target_size=(400, 300),
    batch_size=64,
    class_mode='categorical'
)

Found 682 validated image filenames belonging to 62 classes.


In [ ]:
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Accuracy: {test_accuracy:.4f}')

11/11 [==============================] - 12s 1s/step - loss: 0.5443 - accuracy: 0.8314
Test Accuracy: 0.8314


In [ ]:
'''import pickle

# Save the class indices mapping
with open('/content/drive/My Drive/SRI/class_indices.pkl', 'wb') as file:
    pickle.dump(train_generator.class_indices, file)'''

In [ ]:
# Save the trained model
import joblib
joblib.dump(model, '/content/drive/My Drive/SRI/DenseNet121_model2.pkl')

['/content/drive/My Drive/SRI/DenseNet121_model2.pkl']